In [1]:
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install pillow

In [3]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Directory paths
input_dir = 'input_images'
target_dir = 'target_images'
intermediate_dir = 'intermediate_images'

In [12]:
# Function to load images from a directory
def load_images(image_dir):
    images = []
    for filename in sorted(os.listdir(image_dir)):
        if filename.endswith('.png'):
            img_path = os.path.join(image_dir, filename)
            img = load_img(img_path, target_size=(512, 512))
            img = img_to_array(img)
            img = (img / 127.5) - 1  # Normalize to [-1, 1]
            images.append(img)
    return np.array(images)

# Load input and target images
input_images = load_images(input_dir)
target_images = load_images(target_dir)

# Load intermediate images
intermediate_images = []
for subdir in sorted(os.listdir(intermediate_dir)):
    intermediate_subdir = os.path.join(intermediate_dir, subdir)
    if os.path.isdir(intermediate_subdir):
        images = load_images(intermediate_subdir)
        intermediate_images.append(images)

# Create TensorFlow datasets
input_dataset = tf.data.Dataset.from_tensor_slices(input_images)
target_dataset = tf.data.Dataset.from_tensor_slices(target_images)

# Convert intermediate images list to tuple of datasets
intermediate_datasets = tuple(tf.data.Dataset.from_tensor_slices(steps) for steps in intermediate_images)

# Combine into one dataset
dataset = tf.data.Dataset.zip((input_dataset, intermediate_datasets, target_dataset))

# Shuffle and batch the dataset
batch_size = 32
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size)

# Split into training and validation sets
total_samples = len(input_images) + sum(len(steps) for steps in intermediate_images)
train_size = int(0.8 * total_samples)

# Split into training and validation sets
val_size = total_samples - train_size
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size).take(val_size)

# Print dataset information
# Print dataset information
print("Number of training samples:", train_size)
print("Number of validation samples:", val_size)

Number of training samples: 12
Number of validation samples: 3


In [8]:
# Print lengths of input, intermediate, and target images
print("Number of input images:", len(input_images))
print("Number of intermediate images folders:", len(intermediate_images))
print("Number of target images:", len(target_images))

# Print number of images in each intermediate folder
for i, steps in enumerate(intermediate_images):
    print(f"Number of images in intermediate folder {i+1}:", len(steps))


Number of input images: 3
Number of intermediate images folders: 3
Number of target images: 3
Number of images in intermediate folder 1: 4
Number of images in intermediate folder 2: 4
Number of images in intermediate folder 3: 4


In [13]:
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model

# Function to build the generator (U-Net architecture)
def build_generator(input_shape):
    inputs = Input(shape=input_shape)
    
    # Encoder
    conv1 = Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs)
    conv2 = Conv2D(128, 4, strides=2, padding='same', activation='relu')(conv1)
    conv3 = Conv2D(256, 4, strides=2, padding='same', activation='relu')(conv2)
    conv4 = Conv2D(512, 4, strides=2, padding='same', activation='relu')(conv3)
    
    # Decoder
    deconv1 = Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu')(conv4)
    deconv1_concat = Concatenate()([deconv1, conv3])
    deconv2 = Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu')(deconv1_concat)
    deconv2_concat = Concatenate()([deconv2, conv2])
    deconv3 = Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu')(deconv2_concat)
    deconv3_concat = Concatenate()([deconv3, conv1])
    outputs = Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')(deconv3_concat)
    
    return Model(inputs, outputs)

# Function to build the discriminator (PatchGAN architecture)
def build_discriminator(input_shape):
    inputs = Input(shape=input_shape)
    
    conv1 = Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs)
    conv2 = Conv2D(128, 4, strides=2, padding='same', activation='relu')(conv1)
    conv3 = Conv2D(256, 4, strides=2, padding='same', activation='relu')(conv2)
    conv4 = Conv2D(512, 4, strides=2, padding='same', activation='relu')(conv3)
    outputs = Conv2D(1, 4, strides=1, padding='same', activation='sigmoid')(conv4)
    
    return Model(inputs, outputs)

# Define input shape
input_shape = (256, 256, 3)

# Build generator and discriminator
generator = build_generator(input_shape)
discriminator = build_discriminator(input_shape)

# Display model summaries
generator.summary()
discriminator.summary()



Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │      3,136 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 64, 64,    │    131,200 │ conv2d[0][0]      │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 32, 32,    │    524,544 │ conv2d_1[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 16, 16,    │  2,097,664 │ conv2d_2[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 32, 32,    │  2,097,408 │ conv2d_3[0][0]    │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 512)              │            │ conv2d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 64, 64,    │  1,048,704 │ concatenate[0][0] │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 64, 64,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 256)              │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_2  │ (None, 128, 128,  │    262,208 │ concatenate_1[0]… │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 128, 128,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 128)              │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_3  │ (None, 256, 256,  │      6,147 │ concatenate_2[0]… │
│ (Conv2DTranspose)   │ 3)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 6,171,011 (23.54 MB)

 Trainable params: 6,171,011 (23.54 MB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 64)   │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 64, 64, 128)    │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 32, 32, 256)    │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 16, 16, 512)    │     2,097,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 16, 16, 1)      │         8,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,764,737 (10.55 MB)

 Trainable params: 2,764,737 (10.55 MB)

 Non-trainable params: 0 (0.00 B)